# Simulations of Survival rate

In [35]:
import pandas as pd
import numpy as np
import geopandas as gpd

# Own Classes
#from Simulation_Routing import *
#from Simulating import *

Initialize the class using the ip of the server.

In [ ]:
ip = "35.159.107.60"

In [ ]:
Simulations = Simulation(ip)

## Loop Survivability 

The inputs that have to be given are the decline rate of cfrs, the max number of responders send, the hour the cardiac arrest occured, the aeds that are taken into consideration.

Possible aed filters: 
* ["Yes"] = the truly available aeds
* ["Company"] = aeds that are inside a company/manufactury
* ["Private"] = aeds that are private and couldn't be controlled
* Combination of those, ex.: ["Yes","Company"], ["Private","Company"], ["Yes","Company","Private"], etc.
* None = all aeds in the list are considered

In [ ]:
# Number of times the simulations are run to later get an average survivability rate
NO_SIMULATIONS = 10

# Likelihood of a responder declining the call to action
decline_rate = 0.2
# Maximal number of responder contacted
max_number_responder = 50
# Time of the incident
opening_hour = 12.0
# list of values that are in the available column
filter_values = ["Yes"]
# time of the day
if 8.00 <= opening_hour <= 18.00:
    time_of_day = "day"
else:
    time_of_day = "night"
# proportion of CFR of the total (ex.: 0.01 = 1%)
proportion = 0.01

df_final = pd.DataFrame(columns = ['Patient_loc', 'Responder_loc', 'duration_Responder', 
                                           'Indirect_Responder_loc', 'AED_loc', 'duration_AED',
                                           'Vector_loc', 'duration_Vector'])
x = 0
while x < NO_SIMULATIONS:
    df = Simulations.simulation_run(decline_rate, max_number_responder, opening_hour, filter_values, time_of_day, proportion)
    df_final = pd.concat([df_final,df])
    df_final = df_final.reset_index(drop = True)
    x += 1

1
2
3
4
5
6
7
8
9
10


In [32]:
leuven_submunicipalities = ['Leuven (Heverlee)', 'Leuven (Kessel-Lo)', 'Leuven (Wilsele)', 'Leuven (Wijgmaal)', 'Leuven (Korbeek-Lo)', 'Leuven (Haasrode)', 'Leuven (Leuven)', 'LEUVEN', 'Leuven', 'HAASRODE', 'HEVERLEE', 'KESSEL-LO', 'WILSELE', 'WIJGMAAL', 'KORBEEK-LO']

In [37]:
# try for the cad9 dataset
cad_9 = pd.read_parquet("OneDrive - KU Leuven/Documents/GitHub/AED-Route-Optimization-MDA-Project/Data/cad9.parquet.gzip")

In [38]:
# filter the DataFrame using boolean indexing
filtered_cad_9 = cad_9[cad_9['CityName intervention'].isin(leuven_submunicipalities)]

In [ ]:
P039 - CARDIAAL PROBLEEM (NIET PIJN OP DE BORST),  P008 - PATIËNT MET DEFIBRILLATOR OF PACEMAKER, P003 - HARTSTILSTAND - DOOD - OVERLEDEN

In [62]:
filtered_all = filtered_cad_9[filtered_cad_9["EventType Trip"].isin([
    "P039 - CARDIAAL PROBLEEM (NIET PIJN OP DE BORST)",
    "P008 - PATIËNT MET DEFIBRILLATOR OF PACEMAKER"
])]

In [63]:
filtered_cardiac_arrest = filtered_cad_9[filtered_cad_9["EventType Trip"].isin([
    "P003 - HARTSTILSTAND - DOOD - OVERLEDEN"
])]

In [64]:
print(len(filtered_cardiac_arrest))
print(len(filtered_all))

165
346


In [65]:
import openrouteservice
from openrouteservice import client
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely import geometry
import time
import geopy.distance

import folium
import geopandas as gpd
from shapely.geometry import Point
import random

In [66]:
Check = folium.Map(location=[filtered_all['Latitude intervention'].mean(), filtered_all['Longitude intervention'].mean()], zoom_start=13)
# Add AEDs markers (red)


for _, row in filtered_all.iterrows():
    folium.Marker(
        location=[row['Latitude intervention'], row['Longitude intervention']],
        popup=f"Location",
        icon=folium.Icon(color='green', icon='user', prefix='fa')
    ).add_to(Check)
for _, row in filtered_cardiac_arrest.iterrows():
    folium.Marker(
        location=[row['Latitude intervention'], row['Longitude intervention']],
        popup=f"Location",
        icon=folium.Icon(color='red', icon='user', prefix='fa')
    ).add_to(Check)
# for _, row in AED_check_again.iterrows():
#     folium.Marker(
#         location=[row['latitude'], row['longitude']],
#         popup=f"{row['address']} {row['number']}",
#         icon=folium.Icon(color='orange', icon='heartbeat', prefix='fa')
#     ).add_to(Check)

Check